# AI Singing Notebook 教程

这个笔记本演示完整流程：可选 Demucs 分离 → (可选) 降噪 → HuBERT+DTW 对齐 → 参考原唱的智能混音（含情感包络跟随）。

> 提示：首次跑前先在终端安装依赖 `pip install -e .`，并确保系统有 `ffmpeg`、`rubberband`、`demucs`。

## 0. 环境准备（首次运行）
如已安装可跳过，避免重复下载。

In [ ]:
# 首次运行才需要，重复执行可注释掉
# !pip install -e .


In [ ]:
# 导入与路径设置
from pathlib import Path
import os
import subprocess

from denoise import AIDenoiser
from align import AIAligner
from mixing import AIMixer

PROJECT_ROOT = Path.cwd()
RAW_VOCAL = PROJECT_ROOT / "自己录制（纯人声）.wav"
REF_MIX = PROJECT_ROOT / "原曲（有伴奏）.wav"   # 原曲带伴奏

# Demucs 分离后默认生成的路径（包含空格的文件名也能正常处理）
REF_VOCALS = PROJECT_ROOT / "separated/htdemucs/原曲（有伴奏）/vocals.wav"
INST_AUDIO = PROJECT_ROOT / "separated/htdemucs/原曲（有伴奏）/no_vocals.wav"

# 中间与最终输出
DENOISED_PATH = PROJECT_ROOT / "output_denoised.wav"
ALIGNED_PATH = PROJECT_ROOT / "output_aligned.wav"
FINAL_MIX_PATH = PROJECT_ROOT / "output_final_mix.wav"


## 1. 可选：Demucs 分离原曲（获取伴奏 + 原唱干声）
如果已经有伴奏/原唱干声可跳过。

In [ ]:
if REF_MIX.exists():
    cmd = ["demucs", "-n", "htdemucs", "--two-stems=vocals", str(REF_MIX)]
    print("运行:", " ".join(cmd))
    subprocess.run(cmd, check=True)
    print("分离完成，伴奏/人声在:", INST_AUDIO, REF_VOCALS)
else:
    print("缺少参考音频 REF_MIX:", REF_MIX)


## 2. 可选：AI 降噪（DeepFilterNet）
如果原始录音较干净，可跳过。

In [ ]:
source_vocal = RAW_VOCAL
if RAW_VOCAL.exists():
    print("使用原始人声作为输入:", RAW_VOCAL)
else:
    raise FileNotFoundError(f"找不到人声文件: {RAW_VOCAL}")

# 运行降噪（可根据需要注释掉）
try:
    denoiser = AIDenoiser()
    denoiser.process_file(str(source_vocal), str(DENOISED_PATH))
    source_vocal = DENOISED_PATH
except RuntimeError as e:
    print("跳过降噪:", e)

print("当前对齐/混音使用的人声:", source_vocal)


## 3. 对齐：HuBERT + fastDTW + Rubberband 变速

In [ ]:
align_ref = REF_VOCALS if REF_VOCALS.exists() else REF_MIX
if not align_ref.exists():
    raise FileNotFoundError("找不到参考音频，可先执行 Demucs 分离或指定 REF_MIX")

aligner = AIAligner(safe_mode=True, preprocess=True)
aligner.process_file(str(source_vocal), str(align_ref), str(ALIGNED_PATH))
print("对齐完成 ->", ALIGNED_PATH)


## 4. 智能混音：参考原唱的亮度/响度/情感包络
需提供伴奏 (INST_AUDIO) 与原唱干声 (REF_VOCALS)。

In [ ]:
if not INST_AUDIO.exists() or not REF_VOCALS.exists():
    raise FileNotFoundError("缺少伴奏或原唱干声，请先运行 Demucs 或手动指定路径")

mixer = AIMixer()
mixer.process_mix(
    user_path=str(ALIGNED_PATH),
    inst_path=str(INST_AUDIO),
    ref_path=str(REF_VOCALS),
    output_path=str(FINAL_MIX_PATH),
    enable_emotion=True,
    emotion_strength=1.0,
)
print("混音完成，主输出:", FINAL_MIX_PATH)
print("其他输出: _vocal / _with_ref / _plus_ref")


## 5. 快速试听（可选）
若在 Jupyter 环境支持音频输出，可播放对齐或混音结果。

In [ ]:
from IPython.display import Audio

if ALIGNED_PATH.exists():
    display(Audio(str(ALIGNED_PATH)))
if FINAL_MIX_PATH.exists():
    display(Audio(str(FINAL_MIX_PATH)))


## 6. 常见问题与加速建议
- 采样率不一致：脚本会自动重采样，检测到 ffmpeg 时优先使用 ffmpeg。
- 速度：情感对齐与 Demucs 较耗时，先用短片段验证流程，再跑全长。
- GPU：HuBERT 会自动使用 GPU（如可用），否则退回 CPU。
